In [1]:
#Import Libraries

import os
import cv2
import numpy as np
import pandas as pd
import mahotas as mt
from matplotlib import pyplot as plt
from skimage import io, measure

In [2]:
# Access the dataset folder

AllDatasetDirPath = "C:\\Users\\Hp\\Desktop\\Project\\Test"
FolderList = my_list = os.listdir(AllDatasetDirPath)
print(FolderList) 

In [3]:
def create_dataset():

    names = ['area','perimeter','physiological_length','physiological_width','aspect_ratio','rectangularity','circularity', 'major_axis', 'minor_axis', 'convex_area', 'convex_ratio', \
             'mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b',  \
             'contrast','correlation','inverse_difference_moments','entropy', \
             'classlabel'
            ]
    
    AllDatasetResults=[]

    for i in FolderList:
        
        df = pd.DataFrame([], columns=names)

        print(FolderList.index(i), "/", len(FolderList), "Executing Dataset Name =", i)
        FolderName = i
        DATADIR = AllDatasetDirPath+"/"+FolderName # Outer Folder Path
        # print(DATADIR)

        df = pd.DataFrame([], columns=names)  # Create a new dataframe for each folder


        for dirname, _, filenames in os.walk(DATADIR):
            for filename in filenames:
                os.path.join(dirname, filename)

        CATEGORIES = os.listdir(DATADIR) 
        # print(CATEGORIES)

        for category in CATEGORIES:
            path=os.path.join(DATADIR, category)
            # print(path)
            myclass = CATEGORIES.index(category)
            img_files = os.listdir(path)
            # print(img_files)
            for file in img_files:
                imgpath = path + "\\" + file
                #print(imgpath)
                main_img = cv2.imread(imgpath)


                # Creating target labels
                classlabel = myclass


                # Preprocessing

                # Convert the image to RGB
                img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)

                # Convert the image to Grayscale
                gray = cv2.cvtColor(main_img,cv2.COLOR_BGR2GRAY)
                
                # Perform thresholding
                _, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                
                # Find contours
                contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                # Get the largest contour (assumed to be the leaf)
                leaf_contour = max(contours, key=cv2.contourArea)
                

                # Shape based features
                x, y, w, h = cv2.boundingRect(leaf_contour)
                area = cv2.contourArea(leaf_contour)
                perimeter = cv2.arcLength(leaf_contour, True)
                x, y, w, h = cv2.boundingRect(leaf_contour)
                physiological_length = max(w, h)
                physiological_width = min(w, h)
                aspect_ratio = float(physiological_length) / physiological_width
                rectangularity = area / (physiological_length * physiological_width)
                circularity = (4 * np.pi * area) / (perimeter ** 2)
                convex_area = cv2.contourArea(cv2.convexHull(leaf_contour))
                convex_ratio = area / convex_area


                # Color based features
                b, g, r = cv2.split(img)

                # Calculate mean and standard deviation for each channel
                mean_b = np.mean(b)
                mean_g = np.mean(g)
                mean_r = np.mean(r)

                std_b = np.std(b)
                std_g = np.std(g)
                std_r = np.std(r)
            
                
                # Texture based features
                textures = mt.features.haralick(gray)
                ht_mean = textures.mean(axis=0)
                contrast = ht_mean[1]
                correlation = ht_mean[2]
                inverse_diff_moments = ht_mean[4]
                entropy = ht_mean[8]
                

                vector = [area,perimeter,physiological_length,physiological_width,aspect_ratio,rectangularity,circularity,w,h,convex_area,convex_ratio, \
                        mean_r,mean_g,mean_b,std_r,std_g,std_b,\
                        contrast,correlation,inverse_diff_moments,entropy,\
                        classlabel
                        ]
                
                df_temp = pd.DataFrame([vector],columns=names)
                df = pd.concat([df, df_temp])
                df = df.reset_index(drop=True)

            AllDatasetResults.append(df)

        #Conversion of dataframe to CSV
        df.to_csv(f"{i}.csv")

    return AllDatasetResults
    

In [4]:
dataset = create_dataset()